In [1]:
%load_ext autoreload
%autoreload 2

import gym
from deep_q_learning import *
cartpole_env = gym.make("CartPole-v1").env

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [2]:
agent = DQNAgent(env=cartpole_env)
print("Agent Created")

Agent Created


In [3]:
agent.train_agent(episodes=50000, steps_per_episode=500, batch_size=32)
print("Training Done")

  0%|          | 0/50000 [00:00<?, ?it/s]

episode reward: 13.0 reward of last 100 episodes: 13.0
episode reward: 15.0 reward of last 100 episodes: 14.0


  0%|          | 3/50000 [00:31<146:48:36, 10.57s/it]

episode reward: 22.0 reward of last 100 episodes: 16.666666666666668


  0%|          | 4/50000 [00:51<186:06:02, 13.40s/it]

...

  0%|          | 35/50000 [24:07<2026:21:25, 146.00s/it]

episode reward: 58.0 reward of last 100 episodes: 21.02857142857143


  0%|          | 36/50000 [29:20<2720:36:19, 196.02s/it]

episode reward: 139.0 reward of last 100 episodes: 24.305555555555557


KeyboardInterrupt: 

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
agent.test_agent(episodes=3)